In [ ]:
import cv2
import numpy as np
import mysql.connector
import tensorflow as tf
from win32com.client import Dispatch
from sklearn.preprocessing import LabelEncoder

# Function to connect to the MySQL database
def connect_db():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='Ashu@1996',
        database='myproject'
    )

# Function to fetch eye data from the database
def fetch_eye_data():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT adhar_number, eye FROM eyedata")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return data

# Function to convert image bytes to numpy array
def bytes_to_image(img_bytes):
    nparr = np.frombuffer(img_bytes, np.uint8)
    img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img_np

# Text-to-speech function
def speak(message):
    speaker = Dispatch("SAPI.SpVoice")
    speaker.Speak(message)

# Load the trained CNN model
model = tf.keras.models.load_model('eye_recognition_cnn_model.h5')

# Fetch real user data from MySQL
data = fetch_eye_data()
EYES = []
LABELS = []

for adhaar_number, image in data:
    img_np = bytes_to_image(image)
    resized_img = cv2.resize(img_np, (100, 100))  # Resize image
    EYES.append(resized_img)
    LABELS.append(adhaar_number)

# Normalize image data
EYES = np.array(EYES) / 255.0
LABELS = np.array(LABELS)

# Encode labels
label_encoder = LabelEncoder()
LABELS = label_encoder.fit_transform(LABELS)

# Initialize video capture
video = cv2.VideoCapture(0)
eye_cascade = cv2.CascadeClassifier("c:/users/user/anaconda3/lib/site-packages/cv2/data/haarcascade_eye.xml")

already_recognized = False  # To prevent multiple recognitions

while True:
    ret, frame = video.read()  # Capture frame
    if not ret or frame is None:
        print("Failed to capture frame from camera. Exiting...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)  # Detect eyes
    
    for (x, y, w, h) in eyes:
        crop_img = frame[y:y+h, x:x+w]
        resized_img = cv2.resize(crop_img, (100, 100)) / 255.0  # Normalize the image
        resized_img = np.expand_dims(resized_img, axis=0)  # Expand dimensions for prediction
        
        if not already_recognized:
            predictions = model.predict(resized_img)
            predicted_label = np.argmax(predictions)
            predicted_adhaar = label_encoder.inverse_transform([predicted_label])[0]
            confidence = np.max(predictions)
            
            if confidence > 0.75:  # Confidence threshold for recognition
                print(f"Your Aadhaar Number is: {predicted_adhaar}")
                speak(str(predicted_adhaar))
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.rectangle(frame, (x, y-40), (x+w, y), (0, 255, 0), -1)
                cv2.putText(frame, str(predicted_adhaar), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            else:
                speak("Not Registered")
            
            already_recognized = True
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Your Aadhaar Number is: 813081439046
